In [1]:
import pandas as pd
import catboost as cb
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# лоадим трейн + доп трейн

train = pd.read_csv('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/dataset_ksg.csv')
attr = pd.read_csv('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/attr.csv')

# создаем дикт с задачами, сдвиги по которым будем предсказывать

tasks_ex = pd.read_excel('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/.xlsx')
tasks = pd.Series(tasks_ex['НазваниеЗадачи'].values,index=tasks_ex['Кодзадачи']).to_dict()

In [3]:
# нужно выкинуть все NaN значения по понятным причинам (если тут отсутсвтует либо дата начала, либо
# дата окончания, то предсказание невозможно)

train = train.dropna(subset=['ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи','ДатаначалаБП0', 'ДатаокончанияБП0'])

In [4]:
# переводим строчки в формат даты ДатаначалаБП0

train['ДатаНачалаЗадачи'] = pd.to_datetime(train['ДатаНачалаЗадачи'])
train['ДатаОкончанияЗадачи'] = pd.to_datetime(train['ДатаОкончанияЗадачи'])
train['ДатаначалаБП0'] = pd.to_datetime(train['ДатаначалаБП0'])
train['ДатаокончанияБП0'] = pd.to_datetime(train['ДатаокончанияБП0'])

# переводим в юникс время

train['ДатаНачалаЗадачи'] = train['ДатаНачалаЗадачи'].apply(lambda x: x.timestamp())
train['ДатаОкончанияЗадачи'] = train['ДатаОкончанияЗадачи'].apply(lambda x: x.timestamp())
train['ДатаначалаБП0'] = train['ДатаначалаБП0'].apply(lambda x: x.timestamp())
train['ДатаокончанияБП0'] = train['ДатаокончанияБП0'].apply(lambda x: x.timestamp())

# создаем колонку таргета. Идея такая: предсказываем таргет (= время в секундах от начала до конца задачи)
# переводим секунды в дни, прибавляем к дате начала задачи семпла - получаем ответ


# target перевожу в дни - будем предиктить число дней
# фича plan_time - кол-во дней между плановой датой окончания и начала по договору

#train['fact_time'] = 0
#train['fact_time'] = (train['ДатаОкончанияЗадачи'] - train['ДатаНачалаЗадачи'])/60/60/24
train['plan_time'] = 0
train['plan_time'] = (train['ДатаокончанияБП0'] - train['ДатаначалаБП0'])/60/60/24

train['target']=(train['ДатаОкончанияЗадачи'] - train['ДатаокончанияБП0'])/60/60/24

Всего 28 уникальных объектов и в аттрибутах есть лишние объекты:

In [5]:
print(len(train['obj_key'].unique()))
print(len(attr['obj_key'].unique()))

list_of_objects = train['obj_key'].unique()

28
34


Вытянем из аттрибутов доп фичи:

In [6]:
train['состояние площадки'] = ' '
train['Площадь'] = 0
train['Генпроектировщик'] = 0
train['Генподрядчик'] = 0
train['Кол-во рабочих'] = 0

for obj in list_of_objects:
    train.loc[train['obj_key']==obj,'состояние площадки'] = attr[attr['obj_key']==obj]['состояние площадки'].values[0]
    train.loc[train['obj_key']==obj,'Площадь'] = attr[attr['obj_key']==obj]['Площадь'].values[0]
    train.loc[train['obj_key']==obj,'Генпроектировщик'] = attr[attr['obj_key']==obj]['Генпроектировщик'].values[0]
    train.loc[train['obj_key']==obj,'Генподрядчик'] = attr[attr['obj_key']==obj]['Генподрядчик'].values[0]
    train.loc[train['obj_key']==obj,'Кол-во рабочих'] = attr[attr['obj_key']==obj]['Кол-во рабочих'].values[0]

что не будет использоваться:

Фичи 'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи','ДатаначалаБП0', 'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза', 'date_report' - не используем в модели (первые 4 берем, чтобы получить 2 новых фичи, которые уже заюзаем

In [7]:
train = train[['obj_prg', 'obj_subprg', 'obj_key', 'Кодзадачи',
       'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи',
       'plan_time', 'target', 'состояние площадки', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']]

Много дубликатов (повторяются еженедельно отчеты о том, что типо ничего не сдвинулось == все данные одинаковые)

In [8]:
train.drop_duplicates(inplace = True)

Для каждой из тасок обучим свою модель + поменяем всю фигню, чтобы работало

In [9]:
tasks_list = list(tasks.values())
tasks_list = [x.lower() for x in tasks_list]
tasks_list[5] = 'утверждение перечня тхо в донм'
train['состояние площадки'] = train['состояние площадки'].fillna('')

In [10]:
cat_features = ['obj_prg', 'obj_subprg', 'состояние площадки']

# features - итоговый список фичей!

features = ['obj_prg', 'obj_subprg','состояние площадки', 'ПроцентЗавершенияЗадачи',
       'plan_time', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']

for cat in cat_features:
    train[cat] = train[cat].astype('string')

train['НазваниеЗадачи'] = train['НазваниеЗадачи'].str.lower()    
    
for task in tasks_list:
    model = cb.CatBoostRegressor()
    print(task)
    
    #Создаем трейн и модель только для каждой из таргетных задач
    
    cb_train = train[train['НазваниеЗадачи'] == task]
    
    model.fit(cb_train[features], cb_train['target'], cat_features = cat_features,  verbose = 0)
    model.save_model('CAT_{}.cbm'.format(task))

предпроектные работы
заключение дазу на период строительства
проектно-изыскательские работы
задание на проектирование
разработка и согласование апр
утверждение перечня тхо в донм
разработка проектно-сметной документации (стадия п)
экспертиза проекта
разработка рабочей документации
строительно-монтажные работы
разрешение на строительство
подготовительные работы
земляные работы (котлован)
монолитные работы
кровля
устройство фасадов и остекление (закрытие контутра)
внутренние инженерные системы
технологическое оборудование
отделочные работы
пнр
наружные инженерные сети
благоустройство
завершение строительства и приемка работ
комплексные испытания
проведение итоговой проверки мгсн
получение заключения о соответствии
разрешение на ввод в эксплуатацию
передача объекта на баланс эксплуатирующей организации


In [11]:
# inference 

test_ex = pd.read_excel('/kaggle/input/hackaton-dc/test.xlsx')
test_ex.head()

,№ п/п,obj_prg,obj_subprg,obj_key,obj_shortName,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза
0,36,Образование,Дошкольные учреждения,020-0684,"ДОУ на 125, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-02-01,2020-11-03,2021-12-29,NaN,NaN
1,35,Образование,Дошкольные учреждения,019-0589,"ДОУ на 225, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-05-16,2020-11-03,2021-12-29,NaN,NaN
2,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN
3,89,Культура,Культурные центры,021-0458,"КСЦ ""Печатники"", Полбина",1,Предпроектные работы,100.0,2021-10-12,2023-05-29,2021-10-12,2023-05-12,NaN,NaN
4,34,Образование,Дошкольные учреждения,017-0520,"ДОУ на 350, ул. 6-я Радиальная и ул. Дуговая",1,Предпроектные работы,100.0,2018-11-01,2022-02-15,2018-11-01,2022-02-15,NaN,NaN


In [12]:
#test_ex['obj_key'].iloc[0] = "022-017"

In [13]:
attr.head()

,obj_key,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,date_report
0,022-0170,"Свободна, передана",16402.54,1,2,271.0,2023.01.20
1,022-0526,NaN,18100.00,0,0,NaN,2023.01.20
2,022-0527,Не передана,9850.00,0,0,NaN,2023.01.20
3,022-0522,Не передана,13260.00,0,0,NaN,2023.01.20
4,022-0355,Не передана под снос (для объектов сноса),NaN,1,2,NaN,2023.01.20


In [14]:
merged_test = pd.merge(test_ex, attr, on='obj_key', how='left')
merged_test.head()

,№ п/п,obj_prg,obj_subprg,obj_key,obj_shortName,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих,date_report
0,36,Образование,Дошкольные учреждения,020-0684,"ДОУ на 125, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-02-01,2020-11-03,2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35,Образование,Дошкольные учреждения,019-0589,"ДОУ на 225, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-05-16,2020-11-03,2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN,Занята,17488.0,1.0,2.0,NaN,2023.01.20
3,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN,Занята,17488.0,1.0,2.0,NaN,2023.01.27
4,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN,Занята,17488.0,1.0,2.0,NaN,2023.01.31


In [15]:

merged_test['ДатаНачалаЗадачи'] = pd.to_datetime(merged_test['ДатаНачалаЗадачи'])
merged_test['ДатаОкончанияЗадачи'] = pd.to_datetime(merged_test['ДатаОкончанияЗадачи'])
merged_test['ДатаначалаБП0'] = pd.to_datetime(merged_test['ДатаначалаБП0'])
merged_test['ДатаокончанияБП0'] = pd.to_datetime(merged_test['ДатаокончанияБП0'])

# переводим в юникс время
merged_test['ДатаНачалаЗадачи'] = merged_test['ДатаНачалаЗадачи'].fillna('1970-01-01 00:00:00')
merged_test['ДатаОкончанияЗадачи'] = merged_test['ДатаОкончанияЗадачи'].fillna('1970-01-01 00:00:00')
merged_test['ДатаначалаБП0'] = merged_test['ДатаначалаБП0'].fillna('1970-01-01 00:00:00')
merged_test['ДатаокончанияБП0'] = merged_test['ДатаокончанияБП0'].fillna('1970-01-01 00:00:00')
merged_test['ДатаНачалаЗадачи'] = merged_test['ДатаНачалаЗадачи'].apply(lambda x: x.timestamp())
merged_test['ДатаОкончанияЗадачи'] = merged_test['ДатаОкончанияЗадачи'].apply(lambda x: x.timestamp())
merged_test['ДатаначалаБП0'] = merged_test['ДатаначалаБП0'].apply(lambda x: x.timestamp())
merged_test['ДатаокончанияБП0'] = merged_test['ДатаокончанияБП0'].apply(lambda x: x.timestamp())

# создаем колонку таргета. Идея такая: предсказываем таргет (= время в секундах от начала до конца задачи)
# переводим секунды в дни, прибавляем к дате начала задачи семпла - получаем ответ


# target перевожу в дни - будем предиктить число дней
# фича plan_time - кол-во дней между плановой датой окончания и начала по договору

#merged_test['fact_time'] = 0
#merged_test['fact_time'] = (merged_test['ДатаОкончанияЗадачи'] - merged_test['ДатаНачалаЗадачи'])/60/60/24
merged_test['plan_time'] = 0
merged_test['plan_time'] = (merged_test['ДатаокончанияБП0'] - merged_test['ДатаначалаБП0'])/60/60/24

merged_test['target']=(merged_test['ДатаОкончанияЗадачи'] - merged_test['ДатаокончанияБП0'])/60/60/24

In [16]:
merged_test = merged_test[['obj_prg', 'obj_subprg', 'obj_key', 'Кодзадачи',
       'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи',
       'plan_time', 'target', 'состояние площадки', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']]


In [17]:
merged_test['состояние площадки'] = merged_test['состояние площадки'].fillna('')
cat_features = ['obj_prg', 'obj_subprg', 'состояние площадки']

# features - итоговый список фичей!

features = ['obj_prg', 'obj_subprg','состояние площадки', 'ПроцентЗавершенияЗадачи',
       'plan_time', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']

for cat in cat_features:
    merged_test[cat] = merged_test[cat].astype('string')
    
merged_test['НазваниеЗадачи'] = merged_test['НазваниеЗадачи'].str.lower()  

In [18]:
merged_test.head()

,obj_prg,obj_subprg,obj_key,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,plan_time,target,состояние площадки,Площадь,Генпроектировщик,Генподрядчик,Кол-во рабочих
0,Образование,Дошкольные учреждения,020-0684,1,предпроектные работы,0.0,421.0,34.0,,NaN,NaN,NaN,NaN
1,Образование,Дошкольные учреждения,019-0589,1,предпроектные работы,0.0,421.0,138.0,,NaN,NaN,NaN,NaN
2,Образование,Общеобразовательные учреждения,019-0594,1,предпроектные работы,0.0,225.0,0.0,Занята,17488.0,1.0,2.0,NaN
3,Образование,Общеобразовательные учреждения,019-0594,1,предпроектные работы,0.0,225.0,0.0,Занята,17488.0,1.0,2.0,NaN
4,Образование,Общеобразовательные учреждения,019-0594,1,предпроектные работы,0.0,225.0,0.0,Занята,17488.0,1.0,2.0,NaN


In [19]:
merged_test.shape

(2542512, 13)

In [20]:
merged_test.drop_duplicates(inplace = True)
merged_test.shape

(294759, 13)

In [21]:
tasks_list = list(tasks.values())
tasks_list = [x.lower() for x in tasks_list]
tasks_list[5] = 'утверждение перечня тхо в донм'
models = []
for task in tasks_list:
    model = cb.CatBoostRegressor()
    model.load_model('CAT_{}.cbm'.format(task))

    features = ['obj_prg', 'obj_subprg','состояние площадки', 'ПроцентЗавершенияЗадачи',
    'plan_time', 'Площадь',
    'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']

    # Filter the merged data for the current task
    task_data = merged_test[merged_test['НазваниеЗадачи'] == task]
    
    # Make predictions using the corresponding model
    predictions = model.predict(task_data[features])
    print(task)
#     print(predictions)
    print(predictions.sum())
    # Add the predictions to the merged data
    task_data['predictions'] = predictions
    
    # Do something with the predictions, such as save them to a file or database
    task_data.to_csv('predictions_{}.csv'.format(task), index=False)

предпроектные работы
45365.037607354505
заключение дазу на период строительства
14346.742511248438
проектно-изыскательские работы
33904.320867464456


/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

задание на проектирование
29201.42195649616
разработка и согласование апр
5667.5782172060435
утверждение перечня тхо в донм
4105.720927287734


/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

разработка проектно-сметной документации (стадия п)
23403.12323322205
экспертиза проекта
23986.68888770014
разработка рабочей документации
30109.42354431704


...

/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

проведение итоговой проверки мгсн
31430.153901838334
получение заключения о соответствии
43748.61758023269
разрешение на ввод в эксплуатацию
34189.93220667823


/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_data['predictions'] = predictions
/tmp/ipykernel_20/488546035.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

передача объекта на баланс эксплуатирующей организации
0.0


No objects info loaded
